# TASHO - A model predictive control toolchain for constraint-based task specification of robot motions

#### Presented by: Alejandro Astudillo and Ajay Sathya

Tasho stands for “**TA**sk **S**pecification with receding **HO**rizon control”



| <img src="files/img/overview.svg" width="800"> | 
|:--:| 
| Fig. 1. Tasho's overview |

  
First, we need to import the relevant modules from Tasho and other dependencies (casadi, numpy)

In [1]:
import os
os.chdir(str(os.getcwd())+"/../../")

import sys
from tasho import task_prototype_rockit as tp
from tasho import input_resolution
from tasho import robot as rob
from tasho import environment as env
import casadi as cs
from rockit import MultipleShooting, Ocp
import numpy as np

## Robot model

Import the robot object from the robot's repository

In [2]:
robot_choice = "kinova"
# robot_choice = "yumi"
# robot_choice = "iiwa7"
# robot_choice = "atlas"
# robot_choice = "ur10"

robot = rob.Robot(robot_choice, analytical_derivatives=True)

Loading robot params from json ...
Loaded 7-DoF robot: kinova


The robot object includes functions for forward dynamics, inverse dynamics, forward kinematics, and their derivatives, beside variables for joint limits and barycentric parameters.

#### Forward dynamics:

$
\ddot{\mathbf{q}} = M^{-1}(\mathbf{q})\left(\tau - C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} - G(\mathbf{q})\right)
$

In [3]:
FD = robot.fd
print(FD)

fd:(q[7],q_dot[7],tau[7])->(q_ddot[7]) SXFunction


In [4]:
J_FD = robot.J_fd
print(J_FD)

jac_fd:(q[7],q_dot[7],tau[7])->(jac_fd[7x21]) SXFunction


#### Inverse dynamics:
$
\tau = M(\mathbf{q}) + C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} + G(\mathbf{q})
$

In [5]:
ID = robot.id
print(ID)

id:(q[7],q_dot[7],q_ddot[7])->(tau[7]) SXFunction


In [6]:
J_ID = robot.J_id
print(J_ID)

jac_id:(q[7],q_dot[7],q_ddot[7])->(jac_id[7x21]) SXFunction


#### Forward kinematics:

In [7]:
FK = robot.fk
print(FK)

fk_T:(q[7])->(T_Actuator1[4x4],T_Actuator2[4x4],T_Actuator3[4x4],T_Actuator4[4x4],T_Actuator5[4x4],T_Actuator6[4x4],T_Actuator7[4x4],T_EndEffector[4x4]) SXFunction


#### Upper and lower bound on joint position:

In [8]:
print("Upper bound on q: ",robot.joint_ub)

Upper bound on q:  [inf, 2.2, inf, 2.5656, inf, 2.05, inf]


In [9]:
print("Lower bound on q: ",robot.joint_lb)

Lower bound on q:  [-inf, -2.2, -inf, -2.5656, -inf, -2.05, -inf]


#### Upper and lower bound on joint velocity:

In [10]:
print("Upper bound on q_dot: ",robot.joint_vel_ub)

Upper bound on q_dot:  [0.8727, 0.8727, 0.8727, 0.8727, 0.8727, 0.8727, 0.8727]


In [11]:
print("Lower bound on q_dot: ",robot.joint_vel_lb)

Lower bound on q_dot:  [-0.8727, -0.8727, -0.8727, -0.8727, -0.8727, -0.8727, -0.8727]


#### Upper and lower bound on joint torque:

In [12]:
print("Upper bound on tau: ",robot.joint_torque_ub)

Upper bound on tau:  [39, 39, 39, 39, 9, 9, 9]


In [13]:
print("Lower bound on tau: ",robot.joint_torque_lb)

Lower bound on tau:  [-39, -39, -39, -39, -9, -9, -9]


#### Upper and lower bound on joint acceleration:

In [14]:
print("Upper bound on q_ddot: ",robot.joint_acc_ub)

Upper bound on q_ddot:  None


In [15]:
print("Lower bound on q_ddot: ",robot.joint_acc_lb)

Lower bound on q_ddot:  None


#### Update robot's parameters if needed

In [16]:
ocp_control = "acceleration_resolved"
# ocp_control = "torque_resolved"

if ocp_control == "acceleration_resolved":
    max_joint_acc = 240 * cs.pi / 180
    robot.set_joint_acceleration_limits(lb=[-max_joint_acc]*robot.ndof, ub=[max_joint_acc]*robot.ndof)
    

In [17]:
print("Upper bound on q_ddot: ",robot.joint_acc_ub)

Upper bound on q_ddot:  [4.1887902047863905, 4.1887902047863905, 4.1887902047863905, 4.1887902047863905, 4.1887902047863905, 4.1887902047863905, 4.1887902047863905]


In [18]:
print("Lower bound on q_ddot: ",robot.joint_acc_lb)

Lower bound on q_ddot:  [-4.1887902047863905, -4.1887902047863905, -4.1887902047863905, -4.1887902047863905, -4.1887902047863905, -4.1887902047863905, -4.1887902047863905]


## Task spacification

In [22]:
# Define initial conditions of the robot
if robot_choice == "yumi":
    left_arm_q_init = [-1.35, -3.72e-01, 2.18, 0.78e-01, 2.08, -9.76e-01, -1.71, 1.65e-03, 1.65e-03]
    # Right arm in home configuration
    right_arm_q_init = [0, -2.26, -2.35, 0.52, 0.025, 0.749, 0, 0, 0]

    q_init = np.array(left_arm_q_init + right_arm_q_init).T
elif robot_choice == "kinova":
    q_init = [0, -0.523598, 0, 2.51799, 0, -0.523598, -1.5708]
q_dot_init = [0] * robot.ndof

Select prediction horizon and sample time for the MPC execution

In [23]:
horizon_size = 20
t_mpc = 0.1

Initialize the task context object

In [24]:
tc = tp.task_context(horizon_size * t_mpc, horizon_steps = horizon_size)

Define the input type of the robot (torque or acceleration)

In [25]:
q, q_dot, q_ddot, q0, q_dot0 = input_resolution.acceleration_resolved(tc, robot)

Add object of interest for the robot (in this case a cube + 0.05 m in $z$-direction)

In [26]:
cube_pos = tc.create_expression("cube_pos", "parameter", (3, 1))
T_goal = cs.vertcat(
    cs.hcat([0, 1, 0, cube_pos[0]]),
    cs.hcat([1, 0, 0, cube_pos[1]]),
    cs.hcat([0, 0, -1, cube_pos[2]]),
    cs.hcat([0, 0, 0, 1]),
)

Define constraints at the end of the horizon (final ee position and final joint velocity)

In [27]:
T_ee = robot.fk(q)[7]

final_pos = {
    "hard": False,
    "type": "Frame",
    "expression": T_ee,
    "reference": T_goal,
    "rot_gain": 10,
    "trans_gain": 10,
    "norm": "L1",
}
final_vel = {"hard": True, "expression": q_dot, "reference": 0}
final_constraints = {"final_constraints": [final_pos, final_vel]}
tc.add_task_constraint(final_constraints)

Add penality terms on joint velocity and acceleration for regulatization

In [28]:
tc.add_regularization(
    expression=q_dot, weight=1e-3, norm="L2", variable_type="state", reference=0
)
tc.add_regularization(
    expression=q_ddot, weight=1e-3, norm="L2", variable_type="control", reference=0
)

Set solver and discretization options

In [29]:
tc.set_ocp_solver(
    "ipopt",
    {
        "ipopt": {
            "print_level": 0,
            "tol": 1e-3,
        }
    },
)

In [30]:
disc_settings = {
    "discretization method": "multiple shooting",
    "horizon size": horizon_size,
    "order": 1,
    "integration": "rk",
}
tc.set_discretization_settings(disc_settings)

Set parameter values

In [31]:
tc.ocp.set_value(cube_pos, [0.5, 0, 0.25])
tc.ocp.set_value(q0, q_init)
tc.ocp.set_value(q_dot0, q_dot_init)

Solve the OCP that describes the task

In [32]:
sol = tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  92.00us (  3.41us)  92.00us (  3.41us)        27
       nlp_g  | 421.00us ( 15.59us) 421.63us ( 15.62us)        27
    nlp_grad  |  35.00us ( 35.00us)  34.42us ( 34.42us)         1
  nlp_grad_f  | 144.00us (  5.76us) 146.46us (  5.86us)        25
  nlp_hess_l  | 482.00us ( 20.96us) 481.66us ( 20.94us)        23
   nlp_jac_g  | 593.00us ( 23.72us) 598.73us ( 23.95us)        25
       total  |  51.33ms ( 51.33ms)  56.47ms ( 56.47ms)         1



## MPC Simulation

Create world simulator based on pybullet

In [33]:
from tasho import world_simulator
import pybullet as p

obj = world_simulator.world_simulator()

Add robot to the world environment

In [34]:
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot_choice)

Define world environment

In [35]:
environment = env.Environment()

Add cube to the environment

In [36]:
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")

Add table to the environment

In [37]:
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf = "/models/objects/table.urdf")
environment.add_object(table1, "table1")

Add environment to world simulator

In [38]:
environment.set_in_world_simulator(obj)

Update initial velocity of the cube

In [39]:
cubeID = environment.get_object_ID("cube")
p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.7, 0])

Determine number of samples that the simulation should be executed

In [40]:
no_samples = int(t_mpc / obj.physics_ts)

Correspondence between joint numbers in bullet and OCP

In [41]:
if robot_choice == "yumi":
    joint_indices = [11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9]
elif robot_choice == "kinova":
    joint_indices = [0, 1, 2, 3, 4, 5, 6]

Begin the visualization by applying the initial control signal

In [43]:
ts, q_sol = sol.sample(q, grid="control")
ts, q_dot_sol = sol.sample(q_dot, grid="control")
obj.resetJointState(robotID, joint_indices, q_init)
obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

Execute the MPC loop

In [44]:
for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.5
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos[2] += 0.1  # cube height
    print("Predicted position of cube", predicted_pos)

    # Set parameter values
    tc.ocp.set_value(q0, q_sol[1])
    tc.ocp.set_value(q_dot0, q_dot_sol[1])
    tc.ocp.set_value(cube_pos, predicted_pos)

    # Solve the ocp
    sol = tc.solve_ocp()

    # Sample the solution for the next MPC execution
    ts, q_sol = sol.sample(q, grid="control")
    ts, q_dot_sol = sol.sample(q_dot, grid="control")

    tc.ocp.set_initial(q, q_sol.T)
    tc.ocp.set_initial(q_dot, q_dot_sol.T)

    # Set control signal to the simulated robot
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
    T_ee_sol = robot.fk(q_sol[0])[7]
    pos_ee_sol = T_ee_sol[:3, 3]
    dist_to_cube_sq = cs.sumsqr(pos_ee_sol - predicted_pos)
    print("Distance =",cs.sqrt(dist_to_cube_sq))
    if dist_to_cube_sq <= 4e-2 ** 2:
        break

obj.setController(robotID, "velocity", joint_indices, targetVelocities=[0]*robot.ndof)
obj.run_simulation(1000)

obj.end_simulation()

----------- MPC execution -----------
Predicted position of cube [0.5, 0.29069900142653, 0.45]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 215.00us ( 11.32us) 219.52us ( 11.55us)        19
       nlp_g  |   5.13ms (270.26us)   1.13ms ( 59.40us)        19
    nlp_grad  |  41.00us ( 41.00us)  41.30us ( 41.30us)         1
  nlp_grad_f  | 343.00us ( 17.15us) 344.40us ( 17.22us)        20
  nlp_hess_l  | 976.00us ( 54.22us) 970.65us ( 53.93us)        18
   nlp_jac_g  |   1.43ms ( 71.40us)   1.42ms ( 70.91us)        20
       total  | 172.77ms (172.77ms)  83.42ms ( 83.42ms)         1
Distance = 0.41364454736187
----------- MPC execution -----------
Predicted position of cube [0.5, 1.0321548034461, -1.2361169188309]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  97.00us (  6.93us) 100.17us (  7.15us)        14
       nlp_g  | 589.00us ( 42.07us) 584.16us ( 41.73us)        14
    nlp_grad  |  59.00us ( 59.00us)  59.13us 

Distance = 0.74034064994777
----------- MPC execution -----------
Predicted position of cube [0.49382765704006, 0.40339461330263, 0.12618520754639]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 159.00us ( 11.36us) 163.88us ( 11.71us)        14
       nlp_g  | 821.00us ( 58.64us) 816.14us ( 58.30us)        14
    nlp_grad  |  39.00us ( 39.00us)  39.39us ( 39.39us)         1
  nlp_grad_f  |   4.34ms (289.33us) 331.62us ( 22.11us)        15
  nlp_hess_l  | 685.00us ( 52.69us) 668.44us ( 51.42us)        13
   nlp_jac_g  |   1.17ms ( 78.27us)   1.17ms ( 78.01us)        15
       total  |  97.76ms ( 97.76ms)  47.06ms ( 47.06ms)         1
Distance = 0.37009034265717
----------- MPC execution -----------
Predicted position of cube [0.49410392662072, 0.40074266210388, 0.12511150937398]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  87.00us (  8.70us)  85.33us (  8.53us)        10
       nlp_g  | 400.00us ( 40.00us) 397.57us

Distance = 0.11198656371486
----------- MPC execution -----------
Predicted position of cube [0.49440261628806, 0.39795591232128, 0.12498838998914]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  78.00us (  9.75us)  74.60us (  9.33us)         8
       nlp_g  | 257.00us ( 32.12us) 254.44us ( 31.81us)         8
    nlp_grad  |  40.00us ( 40.00us)  40.39us ( 40.39us)         1
  nlp_grad_f  | 107.00us ( 11.89us) 105.73us ( 11.75us)         9
  nlp_hess_l  | 291.00us ( 41.57us) 294.01us ( 42.00us)         7
   nlp_jac_g  |   4.41ms (490.44us) 413.87us ( 45.99us)         9
       total  |  49.60ms ( 49.60ms)  25.16ms ( 25.16ms)         1
Distance = 0.10113791597667
----------- MPC execution -----------
Predicted position of cube [0.49440261628786, 0.39795591232193, 0.12498838998846]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  59.00us (  7.38us)  60.22us (  7.53us)         8
       nlp_g  | 232.00us ( 29.00us) 230.04us